# Hyperparameter search

### Load Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("Dusduo/1stGen-Pokemon-Images")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

In [ ]:
def get_id2label_and_label2id(dataset):
    """To make it easier for the model  to get the label name from the label id, 
    create a dictionary that maps the label name to an integer and vice versa
    
    Args:
        dataset : Hf dataset to retrieve the labels to ids mapping

    Returns:
        tuple(dict, dict): two dictionary mapping from id to label for the first one,
        and from label to id for the second.
        
    """
    labels = dataset['train'].features['label'].names
    label2id, id2label = dict(),dict()
    
    for idx, label in enumerate(labels):
        label2id[label] = str(idx)
        id2label[str(idx)] = label
        
    return id2label, label2id

In [ ]:
labels = dataset['train'].features['label'].names
id2label, label2id = get_id2label_and_label2id(dataset)

In [ ]:
dataset["train"][750]['image']

In [ ]:
id = dataset["train"][750]['label']
print('Its label:', id)
print('The corresponding pokemon name:', id2label[str(id)])

### Preprocessing

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB").resize((200,200))) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
dataset = dataset.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

### Evaluate

In [ ]:
import evaluate
import numpy as np

f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average="weighted")

### Hyperparameter Search

In [ ]:
from transformers import AutoModelForImageClassification

def model_init():
    return AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
from transformers import Trainer, TrainingArguments
from huggingface_hub import HfFolder


training_args = TrainingArguments(
    output_dir= checkpoint,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_dir=f"{checkpoint}/logs",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    #report_to="tensorboard",
    #push_to_hub=True,
    #hub_strategy="every_save",
    #hub_token=HfFolder.get_token(),
    #hub_model_id=checkpoint
)

# Create a Trainer instance
trainer = Trainer(
    model_init = model_init,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
from ray import tune
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-5,7e-5),
        "per_device_train_batch_size": tune.choice([4,8,16])        
    } 

tune_config = {
    "learning_rate": tune.loguniform(1e-5,7e-5),
    "per_device_train_batch_size": tune.choice([4,8,16]) ,
    "num_train_epochs": tune.choice([3,5,7])        
    }


In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10,
                                        backend="ray",
                                        hp_space=ray_hp_space,
                                        direction="maximize")